<a href="https://colab.research.google.com/github/BorisLoveDev/don-t_know-s_agents/blob/main/new_aug_method_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Импорт необходимых библиотек
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
import numpy as np
from google.colab import drive
import os
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

In [2]:
# Монтирование Google Drive
drive.mount('/content/drive')

# Путь к папке для загрузки данных
data_path = '/content/drive/My Drive/solo_research_aug_cnn/datasets'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Загрузка сохраненных данных
x_train_new = np.load(os.path.join(data_path, 'x_train_new.npy'))
y_train_new = np.load(os.path.join(data_path, 'y_train_new.npy'))
x_val = np.load(os.path.join(data_path, 'x_val.npy'))
y_val = np.load(os.path.join(data_path, 'y_val.npy'))
x_test = np.load(os.path.join(data_path, 'x_test.npy'))
y_test = np.load(os.path.join(data_path, 'y_test.npy'))

# Загрузка дополнительных наборов данных
x_val_easy = np.load(os.path.join(data_path, 'x_val_easy.npy'))
x_val_medium = np.load(os.path.join(data_path, 'x_val_medium.npy'))
x_val_hard = np.load(os.path.join(data_path, 'x_val_hard.npy'))
x_val_indistinguishable = np.load(os.path.join(data_path, 'x_val_indistinguishable.npy'))


# Перемешивание данных
x_val, y_val = shuffle(x_val, y_val, random_state=42)
# Остальные данные уже перемешенные

# Применение one-hot encoding к меткам
y_train_new_ohe = to_categorical(y_train_new, 10)
y_val_ohe = to_categorical(y_val, 10)
y_test_ohe = to_categorical(y_test, 10)

print("Data loaded and one-hot encoded successfully.")

Data loaded and one-hot encoded successfully.


In [4]:
# Печать форм всех загруженных и обработанных данных
print("Форма x_train_new:", x_train_new.shape)
print("Форма y_train_new:", y_train_new.shape)
print("Форма y_train_new_ohe:", y_train_new_ohe.shape)
print("Форма x_val:", x_val.shape)
print("Форма y_val:", y_val.shape)
print("Форма y_val_ohe:", y_val_ohe.shape)
print("Форма x_test:", x_test.shape)
print("Форма y_test:", y_test.shape)
print("Форма y_test_ohe:", y_test_ohe.shape)

# Печать форм дополнительных наборов данных
print("Форма x_val_easy:", x_val_easy.shape)
print("Форма x_val_medium:", x_val_medium.shape)
print("Форма x_val_hard:", x_val_hard.shape)
print("Форма x_val_indistinguishable:", x_val_indistinguishable.shape)

Форма x_train_new: (50000, 28, 28, 1)
Форма y_train_new: (50000,)
Форма y_train_new_ohe: (50000, 10)
Форма x_val: (10000, 28, 28, 1)
Форма y_val: (10000,)
Форма y_val_ohe: (10000, 10)
Форма x_test: (10000, 28, 28, 1)
Форма y_test: (10000,)
Форма y_test_ohe: (10000, 10)
Форма x_val_easy: (10000, 28, 28, 1)
Форма x_val_medium: (10000, 28, 28, 1)
Форма x_val_hard: (10000, 28, 28, 1)
Форма x_val_indistinguishable: (10000, 28, 28, 1)


In [5]:
# Аугментация данных
datagen = ImageDataGenerator(
    rotation_range=10,  # случайный поворот изображений
    width_shift_range=0.1,  # случайный сдвиг изображений по ширине
    height_shift_range=0.1,  # случайный сдвиг изображений по высоте
    zoom_range=0.1  # случайный зум
)
# Подготовка генератора данных
datagen.fit(x_val)

In [6]:
def create_cnn_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        BatchNormalization(),
        Conv2D(64, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(2, 2),
        Dropout(0.25),
        Conv2D(128, (3, 3), activation='relu'),
        BatchNormalization(),
        Dropout(0.25),
        Flatten(),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])

    return model



model_standart_aug = create_cnn_model()

model_new_aug = create_cnn_model()


In [7]:
y_test_ohe.shape

(10000, 10)

In [44]:
# Использование функции для создания модели
model_simple = create_cnn_model()
# Компиляция модели
model_simple.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Каллбеки
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)
early_stop = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)


# Обучение модели на оригинальных данных
history = model_simple.fit(x_val, y_val_ohe,
                           batch_size=32,
                           epochs=50,
                           validation_data=(x_test, y_test_ohe),
                           callbacks=[reduce_lr, early_stop, model_checkpoint])



Epoch 1/50
313/313 [==============================] - 7s 14ms/step - loss: 1.7116 - accuracy: 0.4616 - val_loss: 9.4552 - val_accuracy: 0.1135 - lr: 0.0010
Epoch 2/50
  9/313 [..............................] - ETA: 2s - loss: 1.1821 - accuracy: 0.6215

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


313/313 [==============================] - 3s 9ms/step - loss: 1.0520 - accuracy: 0.6605 - val_loss: 1.0041 - val_accuracy: 0.6814 - lr: 0.0010
Epoch 3/50
313/313 [==============================] - 3s 11ms/step - loss: 0.8099 - accuracy: 0.7310 - val_loss: 0.2385 - val_accuracy: 0.9384 - lr: 0.0010
Epoch 4/50
313/313 [==============================] - 3s 9ms/step - loss: 0.6720 - accuracy: 0.7758 - val_loss: 0.2707 - val_accuracy: 0.9149 - lr: 0.0010
Epoch 5/50
313/313 [==============================] - 4s 12ms/step - loss: 0.5898 - accuracy: 0.8050 - val_loss: 0.1817 - val_accuracy: 0.9393 - lr: 0.0010
Epoch 6/50
313/313 [==============================] - 4s 12ms/step - loss: 0.4840 - accuracy: 0.8425 - val_loss: 0.2566 - val_accuracy: 0.9181 - lr: 0.0010
Epoch 7/50
313/313 [==============================] - 3s 11ms/step - loss: 0.4375 - accuracy: 0.8542 - val_loss: 0.2015 - val_accuracy: 0.9380 - lr: 0.0010
Epoch 8/50
313/313 [==============================] - 3s 11ms/step - loss: 0.

In [45]:
# 1. Оценка точности model_simple на тестовых данных
test_loss, test_acc = model_simple.evaluate(x_test, y_test_ohe)
print("Точность model_simple на тестовых данных:", test_acc)

313/313 [==============================] - 1s 3ms/step - loss: 0.1561 - accuracy: 0.9522
Точность model_simple на тестовых данных: 0.9521999955177307


In [28]:
# 2. Обучение model_standart_aug с аугментацией данных
# Использование функции для создания модели
model_standart_aug = create_cnn_model()
# Компиляция модели
model_standart_aug.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Каллбеки
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)
early_stop = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

history_standart_aug = model_standart_aug.fit(datagen.flow(x_val, y_val_ohe, batch_size=32),
                                              epochs=100,
                                              validation_data=(x_test, y_test_ohe),
                                              callbacks=[reduce_lr, early_stop, model_checkpoint])

Epoch 1/100
313/313 [==============================] - 11s 23ms/step - loss: 0.6811 - accuracy: 0.7875 - val_loss: 5.9580 - val_accuracy: 0.1160 - lr: 0.0010


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 2/100
313/313 [==============================] - 7s 23ms/step - loss: 0.2714 - accuracy: 0.9140 - val_loss: 0.0925 - val_accuracy: 0.9710 - lr: 0.0010
Epoch 3/100
313/313 [==============================] - 7s 22ms/step - loss: 0.2028 - accuracy: 0.9401 - val_loss: 0.0559 - val_accuracy: 0.9817 - lr: 0.0010
Epoch 4/100
313/313 [==============================] - 7s 21ms/step - loss: 0.1652 - accuracy: 0.9498 - val_loss: 0.0479 - val_accuracy: 0.9839 - lr: 0.0010
Epoch 5/100
313/313 [==============================] - 6s 20ms/step - loss: 0.1503 - accuracy: 0.9558 - val_loss: 0.0388 - val_accuracy: 0.9891 - lr: 0.0010
Epoch 6/100
313/313 [==============================] - 8s 25ms/step - loss: 0.1279 - accuracy: 0.9621 - val_loss: 0.0345 - val_accuracy: 0.9883 - lr: 0.0010
Epoch 7/100
313/313 [==============================] - 6s 18ms/step - loss: 0.1247 - accuracy: 0.9632 - val_loss: 0.0342 - val_accuracy: 0.9892 - lr: 0.0010
Epoch 8/100
313/313 [==============================] - 7s 

In [29]:
# 3. Оценка точности model_standart_aug на тестовых данных
test_loss_aug, test_acc_aug = model_standart_aug.evaluate(x_test, y_test_ohe)
print("Точность model_standart_aug на тестовых данных:", test_acc_aug)


313/313 [==============================] - 1s 4ms/step - loss: 0.0156 - accuracy: 0.9955
Точность model_standart_aug на тестовых данных: 0.9955000281333923


In [46]:
# Объединение и обучение model_new_aug
# Объединение наборов данных
x_val_combined = np.concatenate((x_val_medium, x_val_easy))
# Дублирование и объединение меток
y_val_combined = np.tile(y_val_ohe, (2, 1))

# Перемешивание данных
x_val_combined, y_val_combined = shuffle(x_val_combined, y_val_combined, random_state=42)

In [47]:
# 4. Обучение model_new_aug
model_new_aug = create_cnn_model()
# Компиляция модели
model_new_aug.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Каллбеки
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)
early_stop = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)


# Обучение модели на оригинальных данных
history = model_new_aug.fit(x_val_combined, y_val_combined,
                           batch_size=32,
                           epochs=50,
                           validation_data=(x_test, y_test_ohe),
                           callbacks=[reduce_lr, early_stop, model_checkpoint])



Epoch 1/50
625/625 [==============================] - 9s 9ms/step - loss: 0.3587 - accuracy: 0.8881 - val_loss: 0.1138 - val_accuracy: 0.9639 - lr: 0.0010
Epoch 2/50
 10/625 [..............................] - ETA: 3s - loss: 0.2321 - accuracy: 0.9281

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


625/625 [==============================] - 5s 7ms/step - loss: 0.1512 - accuracy: 0.9547 - val_loss: 0.0521 - val_accuracy: 0.9818 - lr: 0.0010
Epoch 3/50
625/625 [==============================] - 6s 10ms/step - loss: 0.1173 - accuracy: 0.9637 - val_loss: 0.0398 - val_accuracy: 0.9872 - lr: 0.0010
Epoch 4/50
625/625 [==============================] - 4s 7ms/step - loss: 0.0876 - accuracy: 0.9715 - val_loss: 0.0600 - val_accuracy: 0.9820 - lr: 0.0010
Epoch 5/50
625/625 [==============================] - 5s 8ms/step - loss: 0.0835 - accuracy: 0.9735 - val_loss: 0.0472 - val_accuracy: 0.9837 - lr: 0.0010
Epoch 6/50
625/625 [==============================] - 6s 9ms/step - loss: 0.0727 - accuracy: 0.9776 - val_loss: 0.0451 - val_accuracy: 0.9845 - lr: 0.0010
Epoch 7/50
625/625 [==============================] - 5s 7ms/step - loss: 0.0628 - accuracy: 0.9794 - val_loss: 0.1005 - val_accuracy: 0.9767 - lr: 0.0010
Epoch 8/50
625/625 [==============================] - 5s 8ms/step - loss: 0.0528

In [48]:
# Оценка точности model_new_aug на тестовых данных
test_loss_new_aug, test_acc_new_aug = model_new_aug.evaluate(x_test, y_test_ohe)
print("Точность model_new_aug на тестовых данных:", test_acc_new_aug)

313/313 [==============================] - 1s 3ms/step - loss: 0.0298 - accuracy: 0.9906
Точность model_new_aug на тестовых данных: 0.9905999898910522


In [33]:
# Использование функции для создания модели
model_new_aug2 = create_cnn_model()
# Компиляция модели
model_new_aug2.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Каллбеки
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)
early_stop = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)
# Обучение model_new_aug на объединенных данных
history_new_aug2 = model_new_aug2.fit(datagen.flow(x_val_combined, y_val_combined, batch_size=32),
                                              epochs=100,
                                              validation_data=(x_test, y_test_ohe),
                                              callbacks=[reduce_lr, early_stop, model_checkpoint])

Epoch 1/100
625/625 [==============================] - 15s 18ms/step - loss: 0.6221 - accuracy: 0.8084 - val_loss: 0.0823 - val_accuracy: 0.9755 - lr: 0.0010
Epoch 2/100
625/625 [==============================] - 13s 20ms/step - loss: 0.2710 - accuracy: 0.9173 - val_loss: 0.0520 - val_accuracy: 0.9830 - lr: 0.0010
Epoch 3/100
625/625 [==============================] - 12s 19ms/step - loss: 0.2116 - accuracy: 0.9355 - val_loss: 0.0518 - val_accuracy: 0.9830 - lr: 0.0010
Epoch 4/100
625/625 [==============================] - 11s 18ms/step - loss: 0.1986 - accuracy: 0.9378 - val_loss: 0.0479 - val_accuracy: 0.9840 - lr: 0.0010
Epoch 5/100
625/625 [==============================] - 13s 20ms/step - loss: 0.1666 - accuracy: 0.9483 - val_loss: 0.0300 - val_accuracy: 0.9895 - lr: 0.0010
Epoch 6/100
625/625 [==============================] - 11s 17ms/step - loss: 0.1538 - accuracy: 0.9521 - val_loss: 0.0460 - val_accuracy: 0.9849 - lr: 0.0010
Epoch 7/100
625/625 [==============================]

In [36]:
# Оценка точности model_new_aug2 на тестовых данных
test_loss_new_aug, test_acc_new_aug = model_new_aug2.evaluate(x_test, y_test_ohe)
print("Точность model_new_aug на тестовых данных:", test_acc_new_aug)

313/313 [==============================] - 1s 3ms/step - loss: 0.0173 - accuracy: 0.9942
Точность model_new_aug на тестовых данных: 0.9941999912261963


In [51]:
def evaluate_model(model, x_val_easy, x_val_medium, x_val_hard, x_val_indistinguishable):
    # Оценка точности на легких данных
    loss_easy, accuracy_easy = model.evaluate(x_val_easy, y_val_ohe)
    print("Точность на легких данных:", accuracy_easy)

    # Оценка точности на средних данных
    loss_medium, accuracy_medium = model.evaluate(x_val_medium, y_val_ohe)
    print("Точность на средних данных:", accuracy_medium)

    # Оценка точности на сложных данных
    loss_hard, accuracy_hard = model.evaluate(x_val_hard, y_val_ohe)
    print("Точность на сложных данных:", accuracy_hard)

    # Оценка точности на данных, которые сложно отличить
    loss_indistinguishable, accuracy_indistinguishable = model.evaluate(x_val_indistinguishable, y_val_ohe)
    print("Точность на данных, которые сложно отличить:", accuracy_indistinguishable)

    return accuracy_easy, accuracy_medium, accuracy_hard, accuracy_indistinguishable




In [56]:
# Пример использования функции
evaluate_model(model_new_aug, x_val_easy, x_val_medium, x_val_hard, x_val_indistinguishable)

313/313 [==============================] - 1s 4ms/step - loss: 9.3510e-04 - accuracy: 0.9999
Точность на легких данных: 0.9998999834060669
313/313 [==============================] - 1s 3ms/step - loss: 0.0012 - accuracy: 1.0000
Точность на средних данных: 1.0
313/313 [==============================] - 1s 3ms/step - loss: 2.0468 - accuracy: 0.5665
Точность на сложных данных: 0.5665000081062317
313/313 [==============================] - 1s 3ms/step - loss: 7.0877 - accuracy: 0.1343
Точность на данных, которые сложно отличить: 0.13429999351501465


(0.9998999834060669, 1.0, 0.5665000081062317, 0.13429999351501465)